#### Imports

In [1]:
import textwrap
import logging
import os
import sys

# Disable logging for the httpx library
logging.getLogger("httpx").disabled = True


In [2]:
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [3]:
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import (
    OTLPSpanExporter as HTTPSpanExporter,
)
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

In [4]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event

In [5]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event


class PrepEvent(Event):
    pass


class InputEvent(Event):
    input: list[ChatMessage]


class ToolCallEvent(Event):
    tool_calls: list[ToolSelection]


class FunctionOutputEvent(Event):
    output: ToolOutput

#### Agent class

In [6]:
from typing import Any, List

from llama_index.core.agent.react import ReActChatFormatter, ReActOutputParser
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.llms.openai import OpenAI


class ReActAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: LLM | None = None,
        tools: list[BaseTool] | None = None,
        extra_context: str | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []

        self.llm = llm or OpenAI()

        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.formatter = ReActChatFormatter(context=extra_context or "")
        self.output_parser = ReActOutputParser()
        self.sources = []

    @step(pass_context=True)
    async def new_user_msg(self, ctx: Context, ev: StartEvent) -> PrepEvent:
        # clear sources
        self.sources = []

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        self.memory.put(user_msg)

        # clear current reasoning
        ctx.data["current_reasoning"] = []

        return PrepEvent()

    @step(pass_context=True)
    async def prepare_chat_history(
        self, ctx: Context, ev: PrepEvent
    ) -> InputEvent:
        # get chat history
        chat_history = self.memory.get()
        current_reasoning = ctx.data.get("current_reasoning", [])
        llm_input = self.formatter.format(
            self.tools, chat_history, current_reasoning=current_reasoning
        )
        return InputEvent(input=llm_input)

    @step(pass_context=True)
    async def handle_llm_input(
        self, ctx: Context, ev: InputEvent
    ) -> ToolCallEvent | StopEvent:
        chat_history = ev.input

        response = await self.llm.achat(chat_history)

        try:
            reasoning_step = self.output_parser.parse(response.message.content)
            ctx.data.get("current_reasoning", []).append(reasoning_step)
            if reasoning_step.is_done:
                self.memory.put(
                    ChatMessage(
                        role="assistant", content=reasoning_step.response
                    )
                )
                return StopEvent(
                    result={
                        "response": reasoning_step.response,
                        "sources": [*self.sources],
                        "reasoning": ctx.data.get("current_reasoning", []),
                    }
                )
            elif isinstance(reasoning_step, ActionReasoningStep):
                tool_name = reasoning_step.action
                tool_args = reasoning_step.action_input
                return ToolCallEvent(
                    tool_calls=[
                        ToolSelection(
                            tool_id="fake",
                            tool_name=tool_name,
                            tool_kwargs=tool_args,
                        )
                    ]
                )
        except Exception as e:
            ctx.data.get("current_reasoning", []).append(
                ObservationReasoningStep(
                    observation=f"There was an error in parsing my reasoning: {e}"
                )
            )

        # if no tool calls or final response, iterate again
        return PrepEvent()

    @step(pass_context=True)
    async def handle_tool_calls(
        self, ctx: Context, ev: ToolCallEvent
    ) -> PrepEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.name: tool for tool in self.tools}

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            if not tool:
                ctx.data.get("current_reasoning", []).append(
                    ObservationReasoningStep(
                        observation=f"Tool {tool_call.tool_name} does not exist"
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                self.sources.append(tool_output)
                ctx.data.get("current_reasoning", []).append(
                    ObservationReasoningStep(observation=str(tool_output))
                )
            except Exception as e:
                ctx.data.get("current_reasoning", []).append(
                    ObservationReasoningStep(
                        observation=f"Error calling tool {tool.metadata.name}: {str(e)}"
                    )
                )

        # prep the next iteration
        return PrepEvent()

#### Basic rag query engine

In [7]:
import data_pull_and_prep.utils as utils
import data_pull_and_prep.data_preparation as data_prep
import textwrap
import basic_rag.rag as rag
from basic_rag.utils import aRetrieveAndAnswer, aRetrieveAndAnswer, RetrieveAndAnswer

/Users/rishikeshdhayarkar/rag-audio-indexing/rag-audio-env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
transcription_with_char_timestamps = utils.import_pkl_file(project_root+"/data/audio_1/ivanka_trump_transcription_char_timestamps.pkl")

custom_chunking_obj = data_prep.CreateCustomTextChunks(transcription_with_char_timestamps)
text_chunks_with_timestamps = custom_chunking_obj.create_custom_text_chunks()

custom_ingestion_obj = rag.CustomRAG(
              index_name="ivanka-08-28-via-class",
              text_chunks_with_timestamps=text_chunks_with_timestamps
              )

await custom_ingestion_obj.create_text_nodes_and_add_to_vector_store()

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Users/rishikeshdhayarkar/rag-audio-indexing/rag-audio-env/lib/python3.12/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
Upserted vectors: 100%|██████████| 42/42 [00:01<00:00, 26.69it/s]


In [9]:
def basic_rag_query_engine(input: str):
    raa = RetrieveAndAnswer(ingestion_obj=custom_ingestion_obj)
    return raa.answer(input)

In [10]:
query_str = "What are some famous quotes mentioned in this podcast and who said them?"
print(textwrap.fill(basic_rag_query_engine(query_str), 80))

Some famous quotes mentioned in this podcast are "Watch the stars and see
yourself running with them" by Marcus Aurelius, "You're under no obligation to
be the same person you were five minutes ago" by Alan Watts, "Radio Gaga" by
Freddie Mercury, and "I torture myself" by Jango Reinhardt.


#### Dual Rag query engine

In [7]:
import react_agent.dual_rag_system as dual_rag_system
import data_pull_and_prep.utils as utils

/Users/rishikeshdhayarkar/rag-audio-indexing/rag-audio-env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [10]:
transcription_with_char_timestamps = utils.import_pkl_file(project_root+"/data/audio_1/ivanka_trump_transcription_char_timestamps.pkl")

In [11]:
class DualRAGManager:
    def __init__(self, transcription_with_char_timestamps):
        self.transcription = transcription_with_char_timestamps
        self.dual_rag = None

    async def setup_dual_rag(self, index_name, neo4j_username, neo4j_password, neo4j_url):
        self.dual_rag = dual_rag_system.DualRAGSystem(
            self.transcription, 
            index_name, 
            neo4j_username, 
            neo4j_password, 
            neo4j_url
        )
        await self.dual_rag.prepare_basic_rag()

    async def query_dual_rag(self, query):
        if not self.dual_rag:
            raise ValueError("DualRAG system not set up. Call setup_dual_rag first.")
        basic_response, graph_response = await self.dual_rag.query_systems(query)        
        return basic_response, graph_response

In [ ]:
# Usage example:
dual_rag_manager = DualRAGManager(transcription_with_char_timestamps)

await dual_rag_manager.setup_dual_rag(
    index_name="ivanka-09-01-agent",
    neo4j_username="neo4j",
    neo4j_password="neo4j_rishi",
    neo4j_url="bolt://localhost:7687"
)

In [13]:
query = "What are some famous quotes mentioned in this podcast and who said them?"
await dual_rag_manager.query_dual_rag(query)

('The famous quotes mentioned in this podcast are:\n\n1. "Watch the stars and see yourself running with them" by Marcus Aurelius.\n2. "You\'re under no obligation to be the same person you were five minutes ago" by Alan Watts.',
 '1. "The only way to make sense out of change is to plunge into it, move with it, and join the dance." - Alan Watts\n2. "Find out who you are and do it on purpose." - Dolly Parton\n3. "When we are no longer able to change a situation, we are challenged to change ourselves." - Victor Frankl\n4. "Stay hungry, stay foolish." - Steve Jobs\n5. "The devil is in the details." - Steve Jobs / Elon Musk\n6. "Everything can be taken from a man but one thing: the last of the human freedoms—to choose one’s attitude in any given set of circumstances, to choose one’s own way." - Victor Frankl\n7. "Love goes very far beyond the physical person of the beloved. It finds its deepest meaning in his spiritual being, his inner self." - Victor Frankl')

#### Agent setup

In [34]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.tools import FunctionTool
import asyncio

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

def sync_dual_rag_query(input: str) -> str:
    return asyncio.run(dual_rag_query(input))

async def dual_rag_query(input: str) -> str:
    basic_response, graph_response = await dual_rag_manager.query_dual_rag(input)
    return f"Basic RAG answer: {basic_response}\nGraph RAG answer: {graph_response}"

dual_rag_tool = FunctionTool(
    fn=sync_dual_rag_query,
    metadata=ToolMetadata(
        name="dual_rag_query_engine",
        description="Provides information about the Ivanka Trump podcast using both basic and graph RAG approaches. Use a plain text question as input to the tool."
    )
)

query_engine_tools = [
    dual_rag_tool
]

In [77]:
agent = ReActAgent(
    llm=OpenAI(model="gpt-3.5-turbo"), tools=query_engine_tools,
    timeout=120,
    verbose=False,
    extra_context="Please use the query engine tool(dual_rag_query_engine) to answer the questions.",
)

ret = await agent.run(input="Hello!")

In [78]:
agent.memory.get()

[ChatMessage(role=<MessageRole.USER: 'user'>, content='Hello!', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="The Ivanka Trump podcast seems to be a personal and reflective discussion where she shares her experiences and challenges while working in the White House and on her father's campaign. Topics covered include her decision not to engage in the politics of the 2024 campaign, her love for her father, admiration for Dolly Parton, experiences as a parent, conflicts between work and family life, the importance of priorities, the role of family support, challenges of getting things done in Washington, importance of helping people, and navigating politics. It offers personal anecdotes and insights into Ivanka Trump's life and experiences in the political world.", additional_kwargs={})]

#### Querying

In [81]:
query_str_suffix = (
    "Always use the query engine tools to answer the questions. "
    "If the tool is not able to answer the question, then give a detailed explanation."
    "Do not remove information from the answer given the tool. Provide elaborate answers while answering the questions."
)

In [82]:
query_str = "What are some famous quotes mentioned in this podcast and who said them?"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

Some famous quotes mentioned in the Ivanka Trump podcast include: 1. "I love my
father very much." - Ivanka Trump 2. "Love is the thing that wins, it usually
does in the end." - Ivanka Trump 3. "No more duels." - Ivanka Trump 4.
"Everything is possible." - Ivanka Trump 5. "If you continue believing that it's
possible, and have that sort of naive notion that you could do it, even if it's
exceptionally difficult, that naive notion turns into some of the greatest
projects ever done." - Ivanka Trump 6. "Watch the stars and see yourself running
with them" - Marcus Aurelius  Additionally, some other quotes mentioned in the
podcast are: 1. "The beauty of life is in the small details, not in big events."
- Marcus Aurelius 2. "When they go low, we go high." - Michelle Obama 3. "I'm
not a businessman; I'm a business, man!" - Jay-Z 4. Alan Watts' quote about
personal growth and change. 5. Victor Frankl's philosophical writings. 6. The
quote by Marcus Aurelius. 7. Joseph Campbell's philosophical w

In [83]:
query_str = "What are Ivanka Trump's thoughts on music?"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

Ivanka Trump's thoughts on music are not explicitly mentioned in the information
available. While she has connections to musicians and musical environments, her
specific opinions or preferences regarding music are not directly discussed in
the podcast.


In [84]:
query_str = "There must be a music related stuff in the context. Give me more details on that."+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

In the Ivanka Trump podcast, music-related topics discussed include Dolly
Parton, live music experiences, Adele's performance at a concert attended by
Ivanka Trump and her kids, Adele's influences like Aretha Franklin, Dolly
Parton's performance at Madison Square Garden, and references to various artists
such as Sinatra, Elvis, Andrew Lloyd Webber, Elton John, and Pavarotti. The
podcast also touches on the enjoyment of seeing artists perform and the
influence of music preferences from Ivanka Trump's father.


In [85]:
query_str = "describe the incident with kim kardashian. Something about prisons"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

In the Ivanka Trump podcast, Kim Kardashian discussed her advocacy for criminal
justice reform, particularly highlighting the case of Alice Marie Johnson.
Johnson was serving a life sentence for a nonviolent drug offense, and
Kardashian's efforts, including lobbying President Trump, led to her clemency.
This collaboration between Kardashian and Ivanka Trump shed light on important
social justice issues within the prison system.


In [86]:
query_str = "What are some architectural projects that Ivanka Trump has worked on?"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

Some of the architectural projects that Ivanka Trump has been involved in
include contributing to the design and construction of iconic structures in New
York City, participating in renovation projects like the Dural hotel and the old
post office building, and engaging in real estate projects alongside her
parents.


In [87]:
query_str = "Describe the impact of NYC on Ivanka Trump's life"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

The impact of New York City (NYC) on Ivanka Trump's life has been significant.
Growing up in NYC exposed her to a fast-paced and dynamic environment, providing
her with opportunities in real estate and fashion. The city also played a role
in shaping her decision to join her father's campaign and work in Washington.
NYC helped her grow and expand her perspective beyond her upbringing on the
upper east side, exposing her to the struggles and experiences of everyday
Americans.


In [88]:
query_str = "What buildings has Ivanka Trump worked on?"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

Ivanka Trump has been involved in various architectural projects, including
contributing to the design and construction of iconic structures in New York
City, participating in renovation projects like the Dural hotel and the old post
office building, and engaging in real estate projects alongside her parents.


In [89]:
query_str = "What does Ivanka Trump say about her children and husband?"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

Ivanka Trump has expressed that her children and husband were everything to her
during her time in Washington. She values the support system provided by her
husband and finds joy and grounding in her children. There are sweet moments
shared, such as her husband making her coffee every morning and her son learning
to make cappuccinos to bring her happiness.


In [90]:
query_str = "Describe the type of work Ivanka Trump did as a senior advisor to the president"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

As a senior advisor to the president, Ivanka Trump worked on various initiatives
related to workforce development, economic empowerment, women's
entrepreneurship, and job creation. She was involved in policy discussions,
meetings with world leaders, and represented the administration on various
domestic and international platforms. Her role also included advocating for
issues such as paid family leave and workforce training programs.


In [91]:
query_str = "Describe the type of work Ivanka Trump did on taxes"+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

Ivanka Trump worked on tax-related initiatives during her time in the White
House, focusing on aspects such as tax reform, economic policies, and advocating
for tax cuts to stimulate economic growth. She was involved in discussions and
efforts to shape tax policies that aimed to benefit American families and
businesses.


In [92]:
query_str = """What unique considerations and complexities were involved in the renovation of the old post 
office building, particularly in terms of layout, room configurations, and preserving the building's historic exterior?"""+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

The renovation of the old post office building involved unique considerations
and complexities such as designing nearly 300 rooms with different layouts,
challenges in moving plumbing due to setbacks in the building, and the priority
of preserving the historic exterior while making gentle additions to maintain
its original beauty. The project required a delicate balance between historical
preservation and modern redevelopment needs, showcasing meticulous planning and
attention to detail.


In [93]:
query_str = """How did Ivanka Trump's children, particularly her son Theo, contribute to her sense of grounding and joy during her time in Washington, D.C.?"""+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

Ivanka Trump's son Theo contributed to her sense of grounding and joy during her
time in Washington, D.C. by learning to make cappuccinos to bring her happiness.
This sweet gesture from her son added a personal touch to her daily routine and
provided moments of joy and connection amidst the busy and demanding environment
of Washington, D.C.


In [94]:
query_str = """What does Ivanka Trump like to do in her free time? What are her hobbies and interests?"""+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

In her free time, Ivanka Trump enjoys spending time with her young children,
volunteering in the community with them, designing city skylines, particularly
in New York City, and appreciating art and beauty in various forms, with a
particular passion for architecture. She also enjoys reading newspapers in bed
while her husband brings her coffee every morning and values spending time with
her family. Additionally, she reads philosophical writings, has a passion for
fashion design, and values engaging in meaningful conversations with various
individuals.


In [96]:
query_str = """Is Jiu-Jitsu discussed in the podcast? If yes give me the details"""+query_str_suffix
ret = await agent.run(input=query_str)
print(textwrap.fill(ret["response"], 80))

Jiu-Jitsu is not specifically discussed in the Ivanka Trump podcast based on the
information available. The podcast mainly focuses on personal anecdotes,
experiences, and reflections related to Ivanka Trump's life, work, and family.
